In [1]:
pwd

'/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/notebooks'

Navigate to py_scripts directory

In [2]:
cd /Users/rachelbrynsvold/dsi/capstone_dir/Capstone/py_scripts/

/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/py_scripts


In [3]:
import os, codecs
import gensim
from gensim import corpora
from collections import defaultdict
from string import punctuation
import pandas as pd
import numpy as np
from gensim.models import ldamodel
from gensim.models import doc2vec
from gensim.models.doc2vec import LabeledSentence
import itertools
import random
from utils_streamers import DirFileMgr, IterFile, CorpStreamer, BOWCorpStreamer

In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

### Enter string used for identifier for dimensional reduction:

In [4]:
small_dr_id_str = "ref7"
dr_id_str = 'ff_50_15p'

Set up needed objects:

In [5]:
fps, small_fps = DirFileMgr(dr_id_str), DirFileMgr(small_dr_id_str)
fps.create_all_dr_fps(new_setup='N')
small_fps.create_all_dr_fps(new_setup='N')

d = corpora.Dictionary.load(fps.dictionary_fp)
corpus = CorpStreamer(d, fps.corp_lst_fp, inc_title='Y')
bow_corpus = BOWCorpStreamer(d, fps.corp_lst_fp, inc_title='Y')

small_d = corpora.Dictionary.load(small_fps.dictionary_fp)
small_corpus = CorpStreamer(small_d, small_fps.corp_lst_fp, inc_title='Y')
small_bow_corpus = BOWCorpStreamer(small_d, small_fps.corp_lst_fp, inc_title='Y')

Which data set will be used? Enter either '5000' or '95': 95
source dir is assigned as ../books/clean/
corpus lst fp is assigned as  ../outputs-git_ignored/ff_50_15p/ff_50_15p_lst.txt
dictionary fp is assigned as  ../outputs-git_ignored/ff_50_15p/ff_50_15p.dict
counts dictionary fp is assigned as  ../outputs-git_ignored/ff_50_15p/ff_50_15p_json.txt
Which data set will be used? Enter either '5000' or '95': 95
source dir is assigned as ../books/clean/
corpus lst fp is assigned as  ../outputs-git_ignored/ref7/ref7_lst.txt
dictionary fp is assigned as  ../outputs-git_ignored/ref7/ref7.dict
counts dictionary fp is assigned as  ../outputs-git_ignored/ref7/ref7_json.txt


In [6]:
class LabeledLineSentence(object):
    def __init__(self, corpus):
        self.corpus = corpus
    def __iter__(self):
        for book in self.corpus:
            yield LabeledSentence(words=book[1:], tags=[book[0]])

In [8]:
labeled_corp = LabeledLineSentence(corpus)
small_labeled_corp = LabeledLineSentence(small_corpus)

In [9]:
next(iter(labeled_corp))[1]

['The Declaration of Independence of the United States of America']

Now I've got a streamed corpus object, and I want to start training a model (doc2vec)

In [10]:
small_doc2vec_model = doc2vec.Doc2Vec()
small_doc2vec_model.build_vocab(small_labeled_corp)

In [21]:
doc2vec_model = doc2vec.Doc2Vec(size=1000, window=8, min_count=5, workers=2, iter=10)
doc2vec_model.build_vocab(labeled_corp)

2017-11-13 22:16:26,214 : INFO : collecting all words and their counts
2017-11-13 22:16:26,348 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-11-13 22:17:53,977 : INFO : PROGRESS: at example #10000, processed 69550298 words (793722/s), 105832 word types, 8551 tags
2017-11-13 22:19:07,420 : INFO : PROGRESS: at example #20000, processed 129634762 words (818381/s), 109285 word types, 17552 tags
2017-11-13 22:20:32,245 : INFO : collected 111490 word types and 25163 unique tags from a corpus of 28492 examples and 193060358 words
2017-11-13 22:20:32,278 : INFO : Loading a fresh vocabulary
2017-11-13 22:20:37,194 : INFO : min_count=5 retains 100958 unique words (90% of original 111490, drops 10532)
2017-11-13 22:20:37,198 : INFO : min_count=5 leaves 193044497 word corpus (99% of original 193060358, drops 15861)
2017-11-13 22:20:38,055 : INFO : deleting the raw counts dictionary of 111490 items
2017-11-13 22:20:38,072 : INFO : sample=0.001 downsamples 0 mo

In [22]:
len(doc2vec_model.docvecs)

25163

In [23]:
len(doc2vec_model.docvecs[0])

1000

In [14]:
tags_lst = [lls[1][0] for lls in labeled_corp]
for ind, tag in enumerate(tags_lst): 
    print(ind, tag)

0 The Declaration of Independence of the United States of America
1 The King James Version of the Bible
2 The Complete Works of William Shakespeare
3 The Magna Carta
4 Apocolocyntosis
5 The House on the Borderland
6 My First Years as a Frenchwoman
7 The Warriors
8 A Voyage to the Moon: With Some Account of the Manners and Customs
9 La Fiammetta
10 Carmilla
11 The Mystery
12 Wild Northern Scenes: Or
13 Divine Comedy
14 The Eulogies of Howard: A Vision
15 365 Foreign Dishes: A Foreign Dish for Every Day in the Year
16 The Mountains of California
17 Punchinello
18 Punchinello
19 Punchinello
20 Punchinello
21 Punchinello
22 Punchinello
23 Punchinello
24 Divine Comedy
25 The Strand Magazine: Vol. 07
26 Tenterhooks
27 White Queen of the Cannibals: the Story of Mary Slessor of Calabar
28 There's Pippins and Cheese to Come
29 Beneath the Banner: Being Narratives of Noble Lives and Brave Deeds
30 Gaslight Sonatas
31 The Mirror of Literature
32 The Triple Alliance
33 Spalding's Official Baseball

544 Pebbles on the shore [by] Alpha of the plough
545 The Reign of Greed
546 Biographical Memorials of James Oglethorpe: Founder of the Colony of Georgia
547 The Story of the Other Wise Man
548 Personal Memoirs of U. S. Grant — Volume 2
549 A History of Freedom of Thought
550 The Camp Fire Girls at Camp Keewaydin; Or
551 Four Short Stories By Emile Zola
552 A Desperate Chance; Or
553 Select Speeches of Kossuth
554 A Life of Gen. Robert E. Lee
555 A Political Diary
556 Sea-Power and Other Studies
557 The Atlantic Monthly
558 The Danger Trail
559 The Aeroplane Boys on the Wing: Or
560 Character Writings of the Seventeenth Century
561 Far from the Madding Crowd
562 The History of England from the First Invasion by the Romans: to the Accession of King George the Fifth; Volume 8
563 The History of Rome
564 The History of Rome
565 The History of Rome
566 The History of Rome
567 The History of Rome
568 The History of Rome (Volumes 1-5)
569 A Christmas Mystery: The Story of Three Wise Men
570 

1119 Notes and Queries
1120 Sonnets by the Nawab Nizamat Jung Bahadur
1121 The Mirror of Literature
1122 The Flood
1123 Virgie's Inheritance
1124 The Tragedy of Othello
1125 A Practical Illustration of "Woman's Right to Labor": A Letter from Marie E. Zakrzewska
1126 The Anti-Slavery Examiner
1127 The Anti-Slavery Examiner
1128 The Anti-Slavery Examiner
1129 The Anti-Slavery Examiner
1130 The Anti-Slavery Examiner
1131 Civil Government in the United States Considered with Some Reference to Its Origins
1132 Egyptian Ideas of the Future Life
1133 Folk-Tales of Napoleon: Napoleonder from the Russian; The Napoleon of the People from the French of Honoré De Balzac
1134 The Slim Princess
1135 The Tragedy of King Lear
1136 Maggie Miller: The Story of Old Hagar's Secret
1137 The Mirror of Literature
1138 The Mirror of Literature
1139 Plays of Gods and Men
1140 Punch
1141 What is Coming? A Forecast of Things after the War
1142 The Tragedy of Macbeth
1143 Emilie the Peacemaker
1144 Travels throug

1629 Punch
1630 A Holiday in the Happy Valley with Pen and Pencil
1631 Hodge and His Masters
1632 The Blood Red Dawn
1633 The Three Sisters
1634 Monkey Jack and Other Stories
1635 Three Plays: The Fiddler's House
1636 The Lair of the White Worm
1637 Ronicky Doone
1638 The Shadow of the North: A Story of Old New York and a Lost Campaign
1639 Colonel Quaritch
1640 Entertaining Made Easy
1641 The Mirror of Literature
1642 The Mirror of Literature
1643 The Mirror of Literature
1644 The Mirror of Literature
1645 Clarissa Harlowe; or the history of a young lady — Volume 7
1646 The Message
1647 Comrades of the Saddle; Or
1648 Gardening for the Million
1649 The Mahabharata of Krishna-Dwaipayana Vyasa Translated into English Prose: Vana Parva
1650 Peace Theories and the Balkan War
1651 Citizen Bird: Scenes from Bird-Life in Plain English for Beginners
1652 On the Indian Sect of the Jainas
1653 Seeing Europe with Famous Authors
1654 A Tramp Abroad
1655 The Jolly Corner
1656 Elizabeth Visits Amer

2077 Overland: A Novel
2078 Brown Wolf and Other Jack London Stories: Chosen and Edited By Franklin K. Mathiews
2079 Dickens in Camp
2080 Organic Syntheses: An Annual Publication of Satisfactory Methods for the Preparation of Organic Chemicals
2081 The Story of the Herschels
2082 Against the Grain
2083 The Nuttall Encyclopædia: Being a Concise and Comprehensive Dictionary of General Knowledge
2084 The Art of Iugling or Legerdemaine: Wherein is Deciphered All the Conueyances of Legerdemaine and Iugling
2085 Sir Robert Hart: The Romance of a Great Career
2086 Friday
2087 A Roman Singer
2088 The Morgesons: A Novel
2089 Richard Vandermarck: A Novel
2090 The Secret City
2091 Captain Fracasse
2092 The International Jewish Cook Book: 1600 Recipes According to the Jewish Dietary Laws with the Rules for Kashering;; the Favorite Recipes of America
2093 The German Classics of the Nineteenth and Twentieth Centuries
2094 Iola Leroy; Or
2095 The Making of Religion
2096 The Constitutional Development

...

26326 Harper's New Monthly Magazine
26327 Arguments before the Committee on Patents of the House of Representatives
26328 Gabrielle of the Lagoon: A Romance of the South Seas
26329 A History of the Reformation (Vol. 1 of 2)
26330 Fiends
26331 The Teaching and Cultivation of the French Language in England during Tudor and Stuart Times: With an Introductory Chapter on the Preceding Period
26332 The Surprises of Life
26333 Camilla; or
26334 Hilda's Mascot: A Tale of "Maryland
26335 Little Tony of Italy
26336 Stories from Virgil
26337 Private Sex Advice to Women: For Young Wives and those who Expect to be Married
26338 Punch
26339 The Iron Ration: Three Years in Warring Central Europe
26340 Punch
26341 British Castles
26342 In the Roar of the Sea
26343 Punch
26344 Punch
26345 The Gentle Shepherd: A Pastoral Comedy
26346 Moonbeams from the Larger Lunacy
26347 The Following of the Star: A Romance
26348 Encyclopaedia Britannica
26349 Notes and Queries
26350 The Life of Johannes Brahms (Vol 1 

26823 The Great Airship: A Tale of Adventure.
26824 John Ermine of the Yellowstone
26825 "I Believe" and other essays
26826 Portage Paths: The Keys of the Continent
26827 Diary of Samuel Pepys — Volume 02: January 1659-1660
26828 To Him That Hath
26829 The Great Strike on the "Q"
26830 Mrs. Maxon Protests
26831 The Arab's Pledge: A Tale of Marocco in 1830
26832 Whither Thou Goest
26833 The Voice from the Void: The Great Wireless Mystery
26834 Sant of the Secret Service: Some Revelations of Spies and Spying
26835 This House to Let
26836 Narcissa
26837 A Book o' Nine Tales.
26838 Diary of Samuel Pepys — Volume 03: February 1659-1660
26839 Chronicles of Dustypore: A Tale of Modern Anglo-Indian Society
26840 Jenifer's Prayer
26841 Animal Proteins
26842 The Speeches (In Full) of the Rt. Hon. W. E. Gladstone
26843 Historical Description of Westminster Abbey
26844 Self-control: A Novel
26845 The Great God Gold
26846 John Jacob Astor
26847 Diary of Samuel Pepys — Volume 04: March/April 1659-16

27312 Quips and Quiddities: A Quintessence of Quirks
27313 The Syndic
27314 Dave Dawson with the R.A.F.
27315 The Pacific Triangle
27316 Samplers and Tapestry Embroideries: Second Edition
27317 Dave Dawson on the Russian Front
27318 White Fire
27319 Diary of Samuel Pepys — Volume 49: January 1666-67
27320 Prisons and Prayer; Or
27321 The Crimson Flash
27322 The Spell of Japan
27323 The Duck-footed Hound
27324 Surgery
27325 Narcissus
27326 The Traditional Games of England
27327 The Traditional Games of England
27328 Kisington Town
27329 Diary of Samuel Pepys — Volume 50: February 1666-67
27330 Dixie After the War: An Exposition of Social Conditions Existing in the South
27331 Scotch Wit and Humor
27332 Records of the Spanish Inquisition
27333 Dürer
27334 The Girl of the Period
27335 The Girl of the Period
27336 Burton of the Flying Corps
27337 A Christmas Carol; Or
27338 Diary of Samuel Pepys — Volume 51: March 1666-67
27339 Ralph Wilton's weird
27340 America
27341 Pintoricchio
27342 Th

27817 The Retrospect
27818 With the Indians in the Rockies
27819 Women in the Printing Trades: A Sociological Study.
27820 In Greek Waters: A Story of the Grecian War of Independence
27821 The Grizzly
27822 Toy-Making at Home: How to Make a Hundred Toys from Odds and Ends
27823 The Great Company: Being a History of the Honourable Company of Merchants-Adventurers Trading into Hudson's Bay
27824 Giordano Bruno
27825 Walt Whitman in Mickle Street
27826 In Beaver World
27827 The San Rosario Ranch
27828 Love Among the Ruins
27829 The Story of Scraggles
27830 The Girl from the Big Horn Country
27831 Abbotsford
27832 The South Pole; an account of the Norwegian Antarctic expedition in the "Fram
27833 A Hundred and Seventy Chinese Poems
27834 The Duchess of Trajetto
27835 The History of Johnny Quæ Genus
27836 Round the Red Lamp: Being Facts and Fancies of Medical Life
27837 Tom Swift and His Motor-Cycle; Or
27838 Poems: Third Edition
27839 The Gist of Japan: The Islands
27840 All about Ferrets 

28311 The Blue Dragon: A Tale of Recent Adventure in China
28312 Cheshire
28313 Fur Farming for Profit
28314 The Influence of the Stars: A book of old world lore
28315 The Duchess of Berry and the Court of Charles X
28316 The Paris Sketch Book of Mr. M. A. Titmarsh; and the Irish Sketch Book
28317 The Expositor's Bible: The First Book of Kings
28318 Recollections of the Civil War: With the Leaders at Washington and in the Field in the Sixties
28319 The Guardians of the Columbia: Mount Hood
28320 The Shadow of Victory: A Romance of Fort Dearborn
28321 Arius the Libyan: A Romance of the Primitive Church
28322 Feline Philosophy
28323 The Magic Egg
28324 The Dominion in 1983
28325 Creatures of the Abyss
28326 Young Blood
28327 China
28328 Great Porter Square: A Mystery. v. 1
28329 Great Porter Square: A Mystery. v. 2
28330 Great Porter Square: A Mystery. v. 3
28331 Dickensian Inns & Taverns
28332 The Collected Writing of Thomas De Quincey
28333 Copyright Law of the United States of America

In [16]:
pandp_ind = 3063
sands_ind = 5418
sherlock_ind = 5885
moby_ind = 4431
scientific_am_ind = 4475

In [25]:
tags_lst[3063]

'Pride and Prejudice'

In [24]:
doc2vec_model.docvecs.most_similar(tags_lst[sands_ind])

2017-11-13 23:41:51,001 : INFO : precomputing L2-norms of doc weight vectors


[('James Lane Allen: A Sketch of his Life and Work', 0.1351635903120041),
 ('Four and Twenty Beds', 0.12506455183029175),
 ('Under the Ensign of the Rising Sun: A Story of the Russo-Japanese War',
  0.11758141964673996),
 ('Stonehenge', 0.11628464609384537),
 ('Strictures on Nullification', 0.11373642086982727),
 ('Races and Immigrants in America', 0.110362209379673),
 ('E. P. Roe: Reminiscences of his Life', 0.1083821952342987),
 ('Under a Charm: A Novel. Vol. II', 0.10682687163352966),
 ('Crooked Trails and Straight', 0.10612525045871735),
 ('Five Mice in a Mouse-trap', 0.10599036514759064)]

In [ ]:
doc2vec_model.

In [104]:
doc2vec_model.docvecs.similarity(tags_lst[13], tags_lst[34])

0.20963378838806776

In [67]:
bible_vec, shakesp_vec = doc2vec_model.docvecs[0], doc2vec_model.docvecs[1]

In [113]:
pandp_vec = doc2vec_model.docvecs[13]
pandp_vec.shape, len(pandp_vec)

((100,), 100)

In [59]:
diff_vec = shakesp_vec - bible_vec

In [63]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import 

In [78]:
twod_bible_vec = bible_vec.reshape(1, 100)
twod_diff_vec = diff_vec.reshape(1, 100)
twod_shakesp_vec = shakesp_vec.reshape(1, 100)
twod_pandp_vec = pandp_vec.reshape(1, 100)

In [84]:
print(twod_bible_vec)

[[ -2.26789992e-03  -3.38749099e-03  -3.39378091e-03  -3.79995978e-03
   -1.34719710e-03  -2.83944677e-03   1.36327301e-03  -4.22230747e-04
    2.53663608e-03   3.49394884e-03   4.26478498e-03   3.92620423e-04
   -4.87274025e-03   1.59396394e-03   3.35374218e-03  -4.21667425e-03
    3.76173807e-03  -3.97574436e-03  -1.46608229e-03   2.07938952e-03
   -3.73467430e-03   3.19301663e-03   4.29960992e-03  -2.19680346e-03
    3.65252933e-03  -2.95786397e-03  -3.17778997e-03  -1.54249952e-03
   -3.93046020e-03  -1.97972800e-03  -3.85790295e-03  -1.71248429e-03
    3.54617630e-04  -1.07043263e-04  -2.39403662e-03  -2.31862185e-03
    3.67591600e-03   1.61199772e-03   1.74636941e-03   4.29156935e-03
    1.45058986e-03  -7.26471771e-05  -4.93741594e-03  -1.30964164e-03
   -1.44652859e-03  -5.52641228e-04   2.33852025e-03   1.61491451e-03
    3.74629488e-03  -2.19661626e-03   3.64151667e-04   4.88952175e-03
   -2.94522732e-04  -3.96283064e-03  -7.84004689e-04  -1.79162039e-03
    3.24501167e-03  

In [79]:
cosine_similarity(twod_pandp_vec, twod_diff_vec)

array([[ 0.02201599]], dtype=float32)

In [115]:
cosine_similarity(twod_diff_vec, twod_pandp_vec)

array([[ 0.02201599]], dtype=float32)

In [136]:
cosine_similarity(twod_diff_vec, twod_shakesp_vec)

array([[ 0.67363441]], dtype=float32)

In [102]:
cosine_similarity(twod_diff_vec, twod_diff_vec)

array([[ 1.]], dtype=float32)

In [105]:
cosine_similarity(doc2vec_model.docvecs[13].reshape(1,100), doc2vec_model.docvecs[34].reshape(1,100))

array([[ 0.2096338]], dtype=float32)

In [134]:
#should probably change this to do the addition/subtraction too... 
#in other words give it two vectors and have the function spit out the closest book
#should I add it on as a method to a doc2vec sub class??
def most_sim_resultant(vec_1, vec2, mode='add' model, tags_lst):
    '''
    Finds the in-set docvec most similar to a docvec resulting from a 'book' addition or subtraction
    '''
    vec_l = len(vec_1)
    
    if mode == 'add':
        resultant_vec = vec_1.reshape(1, vec_l) + vec_2.reshape(1, vec_l)
    elif mode == 'subtract':
        resultant_vec = vec_1.reshape(1, vec_l) - vec_2.reshape(1, vec_l)
    else:
        print("Invalid mode")
        break
                
    cos_sims = [cosine_similarity(resultant_vec, \
        model.docvecs[i].reshape(1, vec_l)) for i in range(len(model.docvecs))]
    
    return tags_lst[np.argmax(cos_sims)], max(cos_sims)

In [135]:
most_sim_resultant(diff_vec, doc2vec_model, tags_lst)

('The Complete Works of William Shakespeare',
 array([[ 0.67363441]], dtype=float32))

In [ ]:
# ^interesting that shakespeare - bible ~= shakespeare...
# ^need to confirm that everything is kosher here

Testing out new objs:

In [12]:
testfmgr = DirFileMgr('ref7')
testfmgr.create_all_dr_fps(new_setup='N')

Which data set will be used? Enter either '5000' or '95': 95
source dir is assigned as ../books/clean/
corpus lst fp is assigned as  ../outputs-git_ignored/ref7/ref7_lst.txt
dictionary fp is assigned as  ../outputs-git_ignored/ref7/ref7.dict
counts dictionary fp is assigned as  ../outputs-git_ignored/ref7/ref7_json.txt


In [13]:
filetest = IterFile(testfmgr.corp_lst_fp)

In [14]:
type(next(iter(filetest)))

str

In [15]:
d = corpora.Dictionary.load(testfmgr.dictionary_fp)
corptest= CorpStreamer(d, testfmgr.corp_lst_fp)

In [16]:
next(iter(corptest))

['testament',
 'version',
 'bible',
 'genesis',
 'firmament',
 'firmament',
 'firmament',
 'herb',
 'herb',
 'firmament',
 'firmament',
 'lesser',
 'firmament',
 'abundantly',
 'fowl',
 'firmament',
 'winged',
 'fowl',
 'fowl',
 'multiply',
 'dominion',
 'fowl',
 'replenish',
 'subdue',
 'dominion',
 'fowl',
 'herb',
 'fowl',
 'herb',
 'seventh',
 'seventh',
 'seventh',
 'sanctified',
 'generations',
 'herb',
 'watered',
 'nostrils',
 'eastward',
 'eden',
 'goeth',
 'eden',
 'mayest',
 'shalt',
 'thereof',
 'shalt',
 'fowl',
 'adam',
 'whatsoever',
 'adam',
 'adam',
 'fowl',
 'adam',
 'adam',
 'cleave',
 'serpent',
 'serpent',
 'doth',
 'sewed',
 'fig',
 'adam',
 'wast',
 'whereof',
 'shouldest',
 'gavest',
 'serpent',
 'beguiled',
 'belly',
 'shalt',
 'shalt',
 'enmity',
 'bruise',
 'shalt',
 'bruise',
 'multiply',
 'shalt',
 'adam',
 'hearkened',
 'shalt',
 'shalt',
 'thorns',
 'shalt',
 'herb',
 'shalt',
 'wast',
 'shalt',
 'adam',
 'adam',
 'eden',
 'adam',
 'eve',
 'gotten',
 'abe

In [12]:
bowcorptest= BOWCorpStreamer(d, testfmgr.corp_lst_fp)

In [13]:
next(iter(bowcorptest))

[(0, 8),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 14),
 (5, 13),
 (6, 3),
 (7, 17),
 (8, 19),
 (9, 2),
 (10, 36),
 (11, 41),
 (12, 2),
 (13, 8),
 (14, 113),
 (15, 53),
 (16, 34),
 (17, 10),
 (18, 6),
 (19, 13),
 (20, 9),
 (21, 127),
 (22, 113),
 (23, 1616),
 (24, 322),
 (25, 20),
 (26, 151),
 (27, 29),
 (28, 16),
 (29, 207),
 (30, 2),
 (31, 41),
 (32, 63),
 (33, 72),
 (34, 73),
 (35, 34),
 (36, 5),
 (37, 26),
 (38, 6),
 (39, 6),
 (40, 77),
 (41, 23),
 (42, 2),
 (43, 21),
 (44, 9),
 (45, 18),
 (46, 11),
 (47, 6),
 (48, 56),
 (49, 191),
 (50, 2),
 (51, 3),
 (52, 113),
 (53, 181),
 (54, 33),
 (55, 34),
 (56, 225),
 (57, 8),
 (58, 1),
 (59, 131),
 (60, 128),
 (61, 16),
 (62, 23),
 (63, 23),
 (64, 3),
 (65, 26),
 (66, 4),
 (67, 66),
 (68, 29),
 (69, 51),
 (70, 25),
 (71, 13),
 (72, 177),
 (73, 74),
 (74, 32),
 (75, 158),
 (76, 3),
 (77, 6),
 (78, 14),
 (79, 15),
 (80, 99),
 (81, 1),
 (82, 77),
 (83, 10),
 (84, 4),
 (85, 36),
 (86, 36),
 (87, 526),
 (88, 2),
 (89, 199),
 (90, 127),
 (91, 2),
 (92, 12